In [1]:
# !pip install tensorflow >> /dev/null
# !pip install tensorflow-hub >> /dev/null
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
bert = hub.KerasLayer('./output/bert_wwm_zh/')

In [4]:
bert(
    tf.ragged.constant([['嗷', '嗷'], ['嗷']]).to_tensor()
)

{'pooled_output': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
 array([[-0.6353977 , -0.70469534,  0.68483686, ...,  0.44020104,
         -0.14942467,  0.80050063],
        [ 0.40159807, -0.7035975 , -0.00531527, ..., -0.19709048,
          0.2169983 ,  0.7176027 ]], dtype=float32)>,
 'sequence_output': <tf.Tensor: shape=(2, 2, 768), dtype=float32, numpy=
 array([[[ 0.20222442,  0.04772816,  0.09283449, ...,  0.45037678,
          -0.11762285,  0.38664433],
         [ 0.21020117,  0.07403799,  0.13659954, ...,  0.4383383 ,
          -0.14299333,  0.41129473]],
 
        [[ 0.38615975,  0.56547326,  0.20320028, ...,  0.782043  ,
          -0.3927768 , -0.00833426],
         [ 0.26134017,  0.60460114,  0.2466737 , ...,  0.7582732 ,
          -0.48962283, -0.08533446]]], dtype=float32)>}

In [6]:
x = [list(x) for x in [
    '我爱你',
    '我恨你',
    '爱你',
    '恨你',
    '爱',
    '恨',
]]
y = [
    1, 0, 1, 0, 1, 0
]

tx = tf.ragged.constant(x).to_tensor()
ty = tf.constant(y)

_input = tf.keras.Input(shape=(None,), dtype=tf.string)
x = _input
x = bert(x)
x = tf.keras.layers.Lambda(lambda x: x['pooled_output'])(x)
x = tf.keras.layers.Dense(1)(x)
x = tf.keras.layers.Activation('sigmoid')(x)

model = tf.keras.Model(inputs=_input, outputs=x)

model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(tx, ty, epochs=10, batch_size=6)
logits = model.predict(tx)
pred = tf.math.round(logits).numpy().flatten().tolist()

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 0.7324 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.5761 - accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4963 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4412 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 0.3990 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 0.3361 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 0.3115 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 0.2899 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 0.2708 - accuracy: 1.0000


In [7]:
print(pred)
print(y)

[1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
[1, 0, 1, 0, 1, 0]


In [9]:
model.save('/tmp/test_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/test_model/assets


In [11]:
tf.keras.models.load_model('/tmp/test_model')(tf.constant([list('我爱你')]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7060459]], dtype=float32)>

In [12]:
tf.keras.models.load_model('/tmp/test_model')(tf.constant([list('恨你')]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.4550731]], dtype=float32)>

In [13]:
tf.keras.models.load_model('/tmp/test_model')(tf.ragged.constant([
    list('我爱你'),
    list('恨你')
]).to_tensor())

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.70604485],
       [0.24683121]], dtype=float32)>